## 데이타 전처리

In [24]:
import os
import pandas as pd
import tensorflow
import numpy as np

In [43]:
train_data_path ="../data/raw_csv/merged_train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,2357,협박 대화,이것 좀 먹어봐\n 이게 머야? 이거 먹으면 나 안괴롭히는거야?\n당연하지 이제 우...
1,4831,일반 대화,"다음 달 여행 계획 세웠어?\n응, 제주도로 3박 4일 일정 잡았어!\n좋겠다! 비..."
2,485,갈취 대화,애 아가야\n네? 어머니\n아범 몰래 나 돈 천만 주라\n어머니 제가 그렇게큰돈이 ...
3,1395,협박 대화,난 널 사랑해\n난 이제 너가 너무 무서워\n무슨소리야? 이쁜 입으로 이쁜말만해 입...
4,140,기타 괴롭힘 대화,아저씨 자리 좀 비켜주시겠어요?\n응.? 뭐때문에 그러는건지 알려줄수있니\n아저씨 ...


In [44]:
train_data.shape

(4882, 3)

In [45]:
#beomi/kcbert-base는 욕설, 위협, 혐오 발언 등 비일상적인 표현이 많은 데이터셋에 특히 적합합니다.
#from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
#tokens = tokenizer("너 정말 죽고 싶냐?", return_tensors='pt')

In [46]:
#형대소 분석기 기반 토크나이저 
# Mecab